In [1]:
!pip install requests

In [2]:
# p.xxx List 7.2

import pathlib
import requests

def download_file(filename):
    prefix = 'https://www1.ncdc.noaa.gov/pub/data/uscrn/products/subhourly01'
    r = requests.get(f"{prefix}/2020/{filename}")
    r.raise_for_status()
    path = pathlib.Path('/home/ubuntu/raw')
    path.mkdir(parents=True, exist_ok=True)
    with open(path / filename, 'wb') as f:
        f.write(r.content)
    print(f"Saved {path / filename}")

FILES = [
    'CRNS0101-05-2020-AK_Aleknagik_1_NNE.txt',
    'CRNS0101-05-2020-AK_Bethel_87_WNW.txt',
]
for filename in FILES:
    download_file(filename)

KeyboardInterrupt: 

In [3]:
!pip install pandas

In [4]:
import glob
import pandas as pd

def read_tables():
    for path in glob.glob('/home/ubuntu/raw/*.txt'):
        yield pd.read_table(path, delimiter='\s+', header=None, dtype='str')

df = pd.concat(read_tables())
df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,23583,20200101,0005,20191231,1505,3,-158.61,59.28,-17.5,0.0,...,C,0,60,0,-99.000,-9999.0,1015,0,1.59,0
1,23583,20200101,0010,20191231,1510,3,-158.61,59.28,-17.5,0.0,...,C,0,60,0,-99.000,-9999.0,1015,0,1.35,0


In [5]:
df1 = df[[0, 1, 2, 8]]
df1.columns = ['wbanno', 'utc_date', 'utc_time', 'temperature']
df1.head(2)

,wbanno,utc_date,utc_time,temperature
0,23583,20200101,0005,-17.5
1,23583,20200101,0010,-17.5


In [6]:
df2 = df1.copy()
df2.index = pd.to_datetime(df2['utc_date'] + df2['utc_time'])
df2.drop(columns=['utc_date', 'utc_time'], inplace=True)
df2.head(2)

,wbanno,temperature
2020-01-01 00:05:00,23583,-17.5
2020-01-01 00:10:00,23583,-17.5


In [7]:
df2.describe().T

,count,unique,top,freq
wbanno,188256,2,26656,94128
temperature,188256,569,0.3,1166


In [8]:
df2['temperature'] = df2['temperature'].astype('float')
df2.describe().T

,count,mean,std,min,25%,50%,75%,max
temperature,188256.0,-9.295505,329.232408,-9999.0,-4.9,4.0,10.3,24.8


In [9]:
df3 = df2.copy()
df3.loc[df3['temperature'] == -9999.0, 'temperature'] = None
df3.describe().T

,count,mean,std,min,25%,50%,75%,max
temperature,188052.0,1.54139,11.669429,-32.0,-4.9,4.0,10.3,24.8
